In [ ]:
from google.colab import files
uploaded = files.upload()


Saving ayurveda.pdf to ayurveda.pdf


In [ ]:
!pip install -q langchain langchain-community pypdf faiss-cpu sentence-transformers transformers accelerate


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


In [ ]:
loader = PyPDFLoader("ayurveda.pdf")
documents = loader.load()
print(f"Total pages loaded: {len(documents)}")



Total pages loaded: 58


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)
print(f"Total chunks created: {len(chunks)}")

Total chunks created: 277


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [ ]:
vectorstore = FAISS.from_documents(chunks, embeddings)


In [ ]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


Device set to use cuda:0


In [ ]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff",
    return_source_documents=True
)


In [ ]:
query = "What is the use case of cumin seeds in disease diagnosis?"

result = rag_chain(query)

print("🔹 ANSWER:\n")
print(result["result"])

print("\n🔹 SOURCES:\n")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"Source {i}:")
    print("Page:", doc.metadata.get("page", "N/A"))
    print(doc.page_content[:500])  # first 500 chars
    print("-" * 80)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


🔹 ANSWER:

It is beneficial for overall bodily strength.  It promotes taste and is vital for the vision of the eyes.  It cures diseases related to Vata, abdominal disorders, Diarrhoea and Vomitting. Malaria.4 The cumin seeds have been studied in the diabetic rats where they are found to significantly reduce blood glucose levels and increase glycosylation in the total haemoglobin within plasma and tissue lipids [5,6]. Additionally, the studies on cumin have reported antinociceptive and anti-neuropathic properties of cuminaldehyde, which is a major constituent of Cuminum cyminum seeds [7]. It might be assistive in improving hunger, vision and bodily strengths. Further, anti -fungal and anti-oxidant  It cures diseases related to Vata, abdominal disorders, Diarrhoea and Vomitting. Malaria.4 The cumin seeds have been studied in the diabetic rats where they are found to significantly reduce blood glucose levels and increase glycosylation in the total haemoglobin within plasma and tissue lipi